In [86]:
%load_ext autoreload
%autoreload 2
 
import sys
sys.path.append('../src')
import xgboost as xgb
import pandas as pd
from features.UserJoin import UserJoin
from features.UserJoin import submit, diff, plt_month, plt_day, load_ids, info, infot
import config as C
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np


from sklearn.model_selection import KFold, StratifiedKFold
import seaborn as sns
from xgboost import plot_tree
import random
from pathlib import Path

import warnings
warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
ds = UserJoin()

# 工具：打分

In [156]:
def get_compare():
    compare = pd.concat([pd.DataFrame({'id': list(C.true_ids), 'label': 1}),
                         pd.DataFrame({'id': list(C.false_ids), 'label': 0}),
                         # pd.DataFrame({'id': list(C.ids_4_2), 'label': .5}),
                         # pd.DataFrame({'id': list(C.ids_4_2_V2), 'label': .5}),
                        #  pd.DataFrame({'id': list(C.ids_13_4), 'label': .692})
                         ])
    compare = compare.set_index('id')
    return compare

def score(ids):
    ids = list(ids)
    compare = get_compare()
    # print(f'共 {len(ids)} 忽略的 ', len(set(ids) - set(compare.index.values)))
    compare = compare.join(pd.DataFrame({'id': ids, 'pred': 1}).set_index('id')).fillna(0)
    compare = compare.reset_index()
    # sns.jointplot(data=compare, x='label', y='pred')
    # print(f'{np.mean((compare.pred - compare.label) ** 2):.3f}')
    print(f'预测了 {len(ids)} 个, 未知/错了 {len(set(ids) - C.true_ids - C.false_ids)} / {(compare.pred != compare.label).sum()}')

    return compare

# 工具：增加数据

In [6]:
# def aug1(df):
#     df = df.copy()
#     df.loc[df.index.isin(testids), 'IS_FLAG'] = 0
#     return df

# 把挖矿用户按比例变化，1个变成10个
def aug2(df):
    idmax = df.index.max()

    d = df[df.IS_FLAG == 1]
    p = d[['pq_f', 'pq_p', 'pq_g', 'pq_z']] * .002

    newdfs = []
    for i in range(-400, 500):
        newd = d.copy()

        newd[['pq_f', 'pq_p', 'pq_g', 'pq_z']] = newd[['pq_f', 'pq_p', 'pq_g', 'pq_z']] + p*i
        newd = newd.reset_index()
        newd.id = newd.id + idmax*(i+6)
        newd = newd.set_index('id')

        newdfs.append(newd)

    return pd.concat(newdfs + [df])

In [45]:
def exp(features, df, params):
    x, x_val = df.loc[df.label != 'test', features], df.loc[df.label == 'test', features]
    y = df.loc[df.label != 'test', 'IS_FLAG']

    model = xgb.XGBClassifier(**params)
    model.fit(x, y)
    pred_y = model.predict(x)
    print((pred_y != y).sum())

    y_val = model.predict(x_val)
    pred_val = pd.DataFrame({'id': x_val.index.values, 'pred': y_val}).groupby('id').sum()
    pred_y = pd.DataFrame({'id': x.index.values, 'ym': df.loc[df.label != 'test'].ym.values, 'pred': pred_y, 'label': y.values})
    return pred_y, pred_val

## 1. 之前 .88 的，不过 UserJoin 里做了一点小改动，所以结果不太一样了

In [120]:
_, pred = exp(C.month_features, ds.month, {})
pred = pred[pred.pred > 7]

score(pred.index.values)
_ = score(load_ids('submit_3_7_1.csv'))

[15:32:42] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0
共预测了 39 个 1。 其中 8 未知
0.190
共预测了 34 个 1。 其中 3 未知
0.238


## 2. 实验：复制了挖矿用户
有那么一点点效果。 扩充测试集之后再看，好像效果也不好了。。。

In [121]:
t = aug2(ds.month)
y, pred = exp(C.month_features, t, {})

[15:32:51] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
183


In [18]:
tmp = pred[pred.pred > 13]
_ = score(tmp.index.values)

共 35 忽略的  7
0.310


## 3. 实验：组合月数据的全局统计
这个尝试下来效果都不太好

In [122]:
user = ds.train[C.user_features]
monthjoin = ds.month.join(user, rsuffix='_mean')

mean_cols = []
for i in monthjoin.columns:
    if 'mean' in i:
        monthjoin[i.replace('mean', 'sub_mean')] = monthjoin[i.replace('_mean', '')] - monthjoin[i]
        mean_cols.append(i)

monthjoin = monthjoin.drop(columns=mean_cols)

monthjoin_features = [
    'pq_f',
    'pq_g',
    'pq_p',
    'pq_z',
    'pf',
    'pg',
    'pp',
    'p_f',
    'p_g',
    'f_g',
    'monthcv',
    # 'pq_f_mean',
    # 'pq_g_mean',
    # 'pq_p_mean',
    # 'pq_z_mean',
    # 'pp_mean',
    # 'pf_mean',
    # 'pg_mean',
    # 'p_f_mean',
    # 'p_g_mean',
    # 'f_g_mean',
    # 'monthcv_mean',

    'pq_f_sub_mean',
    'pq_g_sub_mean',
    'pq_p_sub_mean',
    'pq_z_sub_mean',
    'pp_sub_mean',
    'pf_sub_mean',
    'pg_sub_mean',
    'p_f_sub_mean',
    'p_g_sub_mean',
    'f_g_sub_mean',
    'monthcv_sub_mean',
]

print([i for i in monthjoin.columns if i not in monthjoin_features])
# monthjoin = aug2(monthjoin)
y, pred = exp(monthjoin_features, monthjoin, {})

['ym', '   ', 'ELEC_TYPE_NAME', 'VOLT_NAME', 'PRC_NAME', 'CONTRACT_CAP', 'RUN_CAP', 'SHIFT_NO', 'BUILD_DATE', 'CANCEL_DATE', 'CHK_CYCLE', 'LAST_CHK_DATE', 'TMP_NAME', 'TMP_DATE', 'IS_FLAG', 'label', 'kwh_cal', 'kwh_pap_r2', 'kwh_pap_r3', 'kwh_pap_r4', 'pr2', 'pr3', 'pr4', '2_3', '2_4', '3_4', 'daycv']
[15:33:01] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0


In [33]:
# pred[pred.pred > 0].hist()
tmp = pred[pred.pred > 13]
res = score(tmp.index.values)
# res[res.label != res.pred]

共 37 忽略的  7
0.405


## 4. 实验：尝试一些调参相关的。然后用 gridsearch 试试看。

In [238]:
def fjoin(df):
    user = ds.train[C.user_features]
    monthjoin = df.join(user, rsuffix='_mean')

    mean_cols = []
    for i in monthjoin.columns:
        if 'mean' in i:
            monthjoin[i.replace('mean', 'sub_mean')] = monthjoin[i.replace('_mean', '')] - monthjoin[i]
            mean_cols.append(i)

    monthjoin = monthjoin.drop(columns=mean_cols)

    monthjoin_features = [
        'pq_f',
        'pq_g',
        'pq_p',
        'pq_z',
        'pf',
        'pg',
        'pp',
        'p_f',
        'p_g',
        'f_g',
        'monthcv',
        # 'pq_f_mean',
        # 'pq_g_mean',
        # 'pq_p_mean',
        # 'pq_z_mean',
        # 'pp_mean',
        # 'pf_mean',
        # 'pg_mean',
        # 'p_f_mean',
        # 'p_g_mean',
        # 'f_g_mean',
        # 'monthcv_mean',

        'pq_f_sub_mean',
        'pq_g_sub_mean',
        'pq_p_sub_mean',
        'pq_z_sub_mean',
        'pp_sub_mean',
        'pf_sub_mean',
        'pg_sub_mean',
        'p_f_sub_mean',
        'p_g_sub_mean',
        'f_g_sub_mean',
        'monthcv_sub_mean',
    ]

    return monthjoin, monthjoin_features


def today_exp(params, aug=False, join=False):
    params['use_label_encoder'] = False

    t = ds.month
    # t = ds.month[ds.month.index.isin(ds.train1.index)]
    cols = C.month_features
    if aug:
        t = aug2(t)
    elif join:
        t, cols = fjoin(t)
    pred_y, pred_val = exp(cols, t, params)

    for i in range(10):
        t = pred_y[pred_y.pred == 1]
        t = t.groupby('id').count()
        ids = set(t[t.ym > i].index.values)
        # print(f' > {i}, {len(ids & set(C.minerids))} / {len(ids)}')

        if len(ids) < 10:
            break

    for i in range(22):
        ids = pred_val[pred_val.pred > i].index.values
        if len(ids) > 40:
            continue
        if len(ids) == 0:
            break
        print(f'val: > {i}', end=', ')
        score(ids)

    # return pred_val


In [133]:
today_exp({
    'max_depth': 3,  # 3–10
    'n_estimators': 100,  # 100(lots of observations) to 1000 (few observations)
    'learning_rate': 0.01,  # 0.01~0.3
    'colsample_bytree': 0.5,  # 0.5–1
    'subsample': 0.6  # 0.6–1
})

[15:38:26] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
126
val: > 6, 预测了 40 个 1, 未知/错了 6 / 9
val: > 7, 预测了 36 个 1, 未知/错了 4 / 9
val: > 8, 预测了 31 个 1, 未知/错了 3 / 11
val: > 9, 预测了 25 个 1, 未知/错了 2 / 12


In [239]:
pred_val = today_exp({
    'max_depth': 6,
    'n_estimators': 100,
    'learning_rate': 0.3,
    'colsample_bytree': 1,
    'subsample': 1
})

[16:15:03] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0
val: > 7, 预测了 39 个, 未知/错了 8 / 8
val: > 8, 预测了 34 个, 未知/错了 5 / 8
val: > 9, 预测了 29 个, 未知/错了 3 / 11
val: > 10, 预测了 24 个, 未知/错了 2 / 13
val: > 11, 预测了 23 个, 未知/错了 1 / 13
val: > 12, 预测了 22 个, 未知/错了 1 / 12
val: > 13, 预测了 20 个, 未知/错了 1 / 14
val: > 14, 预测了 17 个, 未知/错了 1 / 13
val: > 15, 预测了 15 个, 未知/错了 1 / 15
val: > 16, 预测了 12 个, 未知/错了 0 / 17
val: > 17, 预测了 11 个, 未知/错了 0 / 18
val: > 18, 预测了 10 个, 未知/错了 0 / 19
val: > 19, 预测了 7 个, 未知/错了 0 / 22
val: > 20, 预测了 5 个, 未知/错了 0 / 24


In [141]:
today_exp({
    'max_depth': 5,  # 3–10
    'n_estimators': 200,  # 100(lots of observations) to 1000 (few observations)
    'learning_rate': 0.05,  # 0.01~0.3
    'colsample_bytree': .8,  # 0.5–1
    'subsample': .8  # 0.6–1
})

[15:41:09] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
45
val: > 7, 预测了 40 个, 未知/错了 7 / 10
val: > 8, 预测了 34 个, 未知/错了 4 / 9
val: > 9, 预测了 29 个, 未知/错了 3 / 11


In [235]:
today_exp({
    'max_depth': 5,  # 3–10
    'n_estimators': 500,  # 100(lots of observations) to 1000 (few observations)
    'learning_rate': 0.05,  # 0.01~0.3
    'colsample_bytree': .8,
    'subsample': .8,
    # 'lambda': 10
}, False, False)

[16:13:35] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
1
val: > 7, 预测了 39 个, 未知/错了 8 / 10
val: > 8, 预测了 35 个, 未知/错了 6 / 8
val: > 9, 预测了 33 个, 未知/错了 4 / 8
val: > 10, 预测了 30 个, 未知/错了 4 / 11
val: > 11, 预测了 28 个, 未知/错了 3 / 10
val: > 12, 预测了 25 个, 未知/错了 2 / 10
val: > 13, 预测了 23 个, 未知/错了 1 / 11
val: > 14, 预测了 21 个, 未知/错了 0 / 12
val: > 15, 预测了 19 个, 未知/错了 0 / 14
val: > 16, 预测了 18 个, 未知/错了 0 / 15
val: > 17, 预测了 14 个, 未知/错了 0 / 17
val: > 18, 预测了 12 个, 未知/错了 0 / 19
val: > 19, 预测了 8 个, 未知/错了 0 / 21
val: > 20, 预测了 6 个, 未知/错了 0 / 23
